In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/'My Drive'/nba_pred/
%ls

/content/drive/My Drive/nba_pred
2019_2020/            cur_ref_set.csv      nba_pred_0.ipynb
append_dataset.ipynb  data_info.txt        nba_pred_1.ipynb
away_data             home_data            reference_data.ipynb
away_scores           home_scores
curr/                 init_data_set.ipynb


In [7]:
#find the new data

import pandas as pd
import numpy as np

season = '2019_2020'
month = 'Mar20'

team_data_trad = pd.read_csv('{}/{}/TeamStatsTrad_{}.csv'.format(season, month, month)).dropna()
team_data_advan = pd.read_csv('{}/{}/TeamStatsAdvan_{}.csv'.format(season, month, month)).dropna()
team_data_misc = pd.read_csv('{}/{}/TeamStatsMisc_{}.csv'.format(season, month, month)).dropna()
team_data_4 = pd.read_csv('{}/{}/TeamStats4_{}.csv'.format(season, month, month)).dropna()
team_data_score = pd.read_csv('{}/{}/TeamStatsScore_{}.csv'.format(season, month, month)).dropna()
team_data_opp = pd.read_csv('{}/{}/TeamStatsOpp_{}.csv'.format(season, month, month)).dropna()
records = pd.read_csv('{}/{}/records_{}.csv'.format(season, month, month)).dropna()


team_name = list(team_data_trad.columns)
team_name.pop(0)

team_stat = {}
for name in team_name:
  team_stat[name] = list(team_data_trad[name])
  team_stat[name]+=(list(team_data_advan[name]))
  team_stat[name]+=(list(team_data_misc[name]))
  team_stat[name]+=(list(team_data_4[name]))
  team_stat[name]+=(list(team_data_score[name]))
  team_stat[name]+=(list(team_data_opp[name]))

#print((team_stat["Los Angeles Lakers"]))
home_score_all = []
away_score_all = []
home_data = np.empty((0, 168))
away_data = np.empty((0, 168))

away_score = list(records['score_away'])
home_score = list(records['score_home'])
home_score_all.extend(home_score)
away_score_all.extend(away_score)

for idx, row in records.iterrows():
  home_team = row['home']
  away_team = row['away']

  data1 = [[*team_stat[home_team], *team_stat[away_team]]]
  data2 = [[*team_stat[away_team], *team_stat[home_team]]]
  home_data = np.append(home_data, list(data1), axis=0)
  away_data = np.append(away_data, list(data2), axis=0)

home_data = np.matrix(home_data)
away_data = np.matrix(away_data)
home_score_all = [x for x in home_score_all if str(x) != 'nan']
away_score_all = [x for x in away_score_all if str(x) != 'nan']
home_score_all = np.reshape(home_score_all, (len(home_score_all), 1))
away_score_all = np.reshape(away_score_all, (len(away_score_all), 1))

print(home_data.shape)
print(home_score_all.shape)

with open("home_data", "ab") as f:
    np.savetxt(f, home_data, fmt='%.2f', delimiter=',')
f.close()

with open("away_data", "ab") as f:
    np.savetxt(f, away_data, fmt='%.2f', delimiter=',')
f.close()

with open("home_scores", "ab") as f:
    np.savetxt(f, home_score_all, fmt='%f', delimiter=',')
f.close()

with open("away_scores", "ab") as f:
    np.savetxt(f, away_score_all, fmt='%f', delimiter=',')
f.close()

(78, 168)
(78, 1)
